In [2]:
import pandas as pd
import ipywidgets as widgets
import base64

In [4]:
# reading csv
df = pd.read_csv("https://drive.google.com/uc?export=download&id=1ITLYTy36uuZTALDX42jZiCTn8LZV-YEj")

In [6]:
item_tot_stock = df.groupby("item_id").stock_quantity.sum()
low_stock_items = item_tot_stock[item_tot_stock < 5].index
mask = df.item_id.isin(low_stock_items)
low_stock_df = df[mask]

# low_stock_df

In [8]:
def render_boxes_content(item_group):
    item_id = item_group.item_id.iloc[0]  # first (all the same)
    image_url = item_group.image.iloc[0]  # first (all the same)

    # left column
    col1_html = f'<h1>{item_id}</h1><br><img src="{image_url}"></img>'
    column1 = widgets.HTML(col1_html)

    # right column
    col2_rows = []
    for _, row in item_group.iterrows():
        name = f"{row.category} {row.color_long} {row['size']}"
        col2_row = widgets.HBox(
            [
                widgets.Label(name),
                widgets.IntText(row.stock_quantity),
                widgets.Label(str(row.price)),
            ]
        )
        col2_rows.append(col2_row)

    column2 = widgets.VBox(col2_rows, layout=widgets.Layout(padding="30px"))

    # combining both
    return widgets.HBox([column1, column2])


# render_boxes_content(boxes_content[15])

In [10]:
class InventoryDashboard:
    def __init__(self, df):
        self.df = df.copy()
        self.out = widgets.Output()
        display(self.out)
        self.start_idx = 0
        self.page_step = 2
        self.boxes = []

        self.define_header()
        self.render()

    def get_n_group(self, n):
        item_id = self.df.item_id.unique()[n]
        return self.df.groupby("item_id").get_group(item_id)

    def store_values(self):
        for box in dash.boxes:
            values = []
            for row in box.children[1].children:
                values.append(row.children[1].value)
            self.df.loc[box.idx, "stock_quantity"] = values

    def define_header(self):
        prev_btn = widgets.Button(description="Prev")
        next_btn = widgets.Button(description="Next")
        export_btn = widgets.Button(description="Export Order", button_style="danger")
        page_idx_lbl = widgets.Label(f"{self.start_idx} - {self.start_idx + self.page_step}")

        def _prev(b):
            self.store_values()
            self.start_idx -= self.page_step
            if self.start_idx < 0:
                self.start_idx = 0
            page_idx_lbl.value = f"{self.start_idx} - {self.start_idx + self.page_step}"
            self.render()

        def _next(b):
            self.store_values()
            self.start_idx += self.page_step
            page_idx_lbl.value = f"{self.start_idx} - {self.start_idx + self.page_step}"
            self.render()

        def _export(b):
            self.store_values()
            self.create_download_url()

        prev_btn.on_click(_prev)
        next_btn.on_click(_next)
        export_btn.on_click(_export)
        self.header = widgets.HBox([prev_btn, next_btn, export_btn, page_idx_lbl])

    def get_rows(self):
        self.boxes = []
        for n in range(self.start_idx, self.start_idx + self.page_step):
            item_group = self.get_n_group(n)
            box = render_boxes_content(item_group)
            box.idx = item_group.index
            self.boxes.append(box)

        vbox_content = [self.header] + self.boxes
        return widgets.VBox(vbox_content)

    def render(self):
        self.out.clear_output(wait=True)
        with self.out:
            display(self.get_rows())

    def create_download_url(self):
        self.out.clear_output(wait=True)
        dwld = widgets.HTML(value="")

        fp = "export.csv"
        self.df.to_csv(fp)

        with open(fp, "rb") as f:
            data = f.read()
        b64 = base64.b64encode(data)
        payload = b64.decode()

        dwld.value = (
            f'<a download="{fp}" href="data:text/csv;base64,{payload}" target="_blank">Click here to download</a>'
        )

        with self.out:
            display(dwld)

In [12]:
dash = InventoryDashboard(df=low_stock_df)

Output()